## 3(1) Feature Generation

### Import

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import datetime
from sklearn.decomposition import PCA
from haversine import haversine 
# import random
# from sklearn.cluster import AffinityPropagation  # not scalable data(Memory Error)
# from sklearn.cluster import DBSCAN
# from sklearn.cluster import MiniBatchKMeans

import warnings ; warnings.filterwarnings('ignore')

### Read Data

In [2]:
train = pd.read_parquet('../data/raw/train_clean.parquet')
test = pd.read_parquet('../data/raw/test_clean.parquet')

### $\blacktriangleright$ Base Feature

In [3]:
# 불필요한 열은 삭제한다.
del train['vehicle_restricted'],train['height_restricted']
del test['vehicle_restricted'],test['height_restricted']

# 보류열도 제외한다.
del train['road_name'], train['start_node_name'], train['end_node_name']
del test['road_name'], test['start_node_name'], test['end_node_name']

In [4]:
# base_date를 날짜로 변경한다.
train['base_date'] = pd.to_datetime(train['base_date'], format='%Y%m%d')
test['base_date'] = pd.to_datetime(test['base_date'], format='%Y%m%d')

In [5]:
# Category Encoding을 수행한다.
str_col = ['day_of_week','start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

### $\blacktriangleright$ Feature Generation

In [6]:
feature_train, feature_test = train.copy(), test.copy()

- Date Information

In [7]:
# 월별 해당 주차를 알려주는 함수를 정의한다.
def month_week(DATE):
    standard = datetime.datetime(DATE.year,DATE.month,1)
    if standard.weekday() == 6:
        no = DATE.weekofyear - standard.isocalendar()[1] + 1
    else:
        no = DATE.weekofyear - standard.isocalendar()[1]
    return no

In [8]:
feature_train['base_year'] = train['base_date'].dt.year
feature_test['base_year'] = test['base_date'].dt.year

In [9]:
feature_train['base_month'] = train['base_date'].dt.month
feature_test['base_month'] = test['base_date'].dt.month

In [10]:
feature_train['base_day'] = train['base_date'].dt.day
feature_test['base_day'] = test['base_date'].dt.day

In [11]:
feature_train['base_no_week'] = [month_week(DATE) for DATE in train['base_date']]
feature_test['base_no_week'] = [month_week(DATE) for DATE in test['base_date']]

In [12]:
# 중복을 막기 위해 base_date는 삭제한다.
del feature_train['base_date'], feature_test['base_date']

- Transform Coordinate

In [13]:
pca = PCA(random_state=2022).fit(np.vstack((train[['start_latitude','start_longitude']].values,
                                            train[['end_latitude','end_longitude']].values)))

feature_train[['pca_start_latitude', 'pca_start_longitude']] = pca.transform(train[['start_latitude','start_longitude']])
feature_train[['pca_end_latitude', 'pca_end_longitude']] = pca.transform(train[['end_latitude','end_longitude']])

feature_test[['pca_start_latitude', 'pca_start_longitude']] = pca.transform(test[['start_latitude','start_longitude']])
feature_test[['pca_end_latitude', 'pca_end_longitude']] = pca.transform(test[['end_latitude','end_longitude']])

In [13]:
# cluster학습시 MemoryError가 발생하여 random하게 100000로 학습을 시도한다.
# sample = random.sample(coordinate.tolist(), 100000)
# cluster = AffinityPropagation(random_state=2022).fit(sample)

- Calculate distance<br>
  - 구에서의 두 점 사이 거리를 구하는 방식인 `haversine`은 package를 사용해 구한다.<br>
    [harversine] https://en.wikipedia.org/wiki/Haversine_formula<br>
    [harversine package] https://pypi.org/project/haversine/
  - 거리를 재는 `Spherical Law of Cosines`, `Equirectangular approximation`과 방위각을 재는 `bearing`은 아래의 식으로 구한다.<br>
    [etc] https://www.movable-type.co.uk/scripts/latlong.html
  - [국토지리원 산출법] https://linuxism.ustd.ip.or.kr/831
  - 이외 거리를 재는 방법으로 유클라디안, 맨하탄, 체비셰프 등이 있으나 평면상의 거리라 유효할 지 알 수 없다.<br>
    PCA를 통해 평면으로 변환한 좌표는 민코우스키로 유클라디안, 맨하탄 모두 구할 수 있다.

In [14]:
# 구면 코사인 법칙(meters)
def spheical_cosines(LAT1, LAT2, LONG1, LONG2):
    lat1, lat2, d_long = map(np.radians, (LAT1, LAT2, LONG2-LONG1))
    x = np.sin(lat1) * np.sin(lat2) + np.cos(lat1) * np.cos(lat2) * np.cos(d_long)
    return np.arccos(x)*6371e3   

# 정방향 근사(meters)
def approximation(LAT1, LAT2, LONG1, LONG2):
    d_lat, m_lat, d_long = map(np.radians, (LAT2-LAT1, (LAT1+LAT2)/2, LONG2-LONG1))
    x = d_long * np.cos(m_lat)
    y = d_lat
    return 6371e3 * np.sqrt(x**2+y**2)

# 방위각
def bearing(LAT1, LAT2, LONG1, LONG2):
    lat1, lat2, d_long = map(np.radians, (LAT1, LAT2, LONG2-LONG1))
    y = np.sin(d_long) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(d_long)
    return np.degrees(np.arctan2(y,x))

In [15]:
feature_train['harversine'] = [haversine((LAT1,LONG1),(LAT2,LONG2), unit='m') 
                               for _,LAT1,LONG1,LAT2,LONG2 in train[['start_latitude','start_longitude','end_latitude','end_longitude']].itertuples()]
feature_test['harversine'] = [haversine((LAT1,LONG1),(LAT2,LONG2), unit='m') 
                              for _,LAT1,LONG1,LAT2,LONG2 in test[['start_latitude','start_longitude','end_latitude','end_longitude']].itertuples()]

In [16]:
feature_train['cosine'] = spheical_cosines(train['start_latitude'],train['end_latitude'], train['start_longitude'],train['end_longitude'])
feature_test['cosine'] = spheical_cosines(test['start_latitude'],test['end_latitude'], test['start_longitude'],test['end_longitude'])

In [17]:
feature_train['approximation'] = approximation(train['start_latitude'],train['end_latitude'], train['start_longitude'],train['end_longitude'])
feature_test['approximation'] = approximation(test['start_latitude'],test['end_latitude'], test['start_longitude'],test['end_longitude'])

In [18]:
feature_train['bearing'] = bearing(train['start_latitude'],train['end_latitude'], train['start_longitude'],train['end_longitude'])
feature_test['bearing'] = bearing(test['start_latitude'],test['end_latitude'], test['start_longitude'],test['end_longitude'])

In [19]:
# 민코우스키

In [19]:
feature_train['m_latitude'] = train[['start_latitude','end_latitude']].mean(axis=1)
feature_train['m_longitude'] = train[['start_longitude','end_longitude']].mean(axis=1)
feature_test['m_latitude'] = test[['start_latitude','end_latitude']].mean(axis=1)
feature_test['m_longitude'] = test[['start_longitude','end_longitude']].mean(axis=1)

In [20]:
feature_train['m_pca_latitude'] = feature_train[['pca_start_latitude','pca_end_latitude']].mean(axis=1)
feature_train['m_pca_longitude'] = feature_train[['pca_start_longitude','pca_end_longitude']].mean(axis=1)
feature_test['m_pca_latitude'] = feature_test[['pca_start_latitude','pca_end_latitude']].mean(axis=1)
feature_test['m_pca_longitude'] = feature_test[['pca_start_longitude','pca_end_longitude']].mean(axis=1)

### Confirm Data

In [21]:
feature_train

,id,day_of_week,base_hour,road_in_use,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,weight_restricted,...,pca_end_latitude,pca_end_longitude,harversine,cosine,approximation,bearing,m_latitude,m_longitude,m_pca_latitude,m_pca_longitude
0,TRAIN_0000000,1,17,0,1,106,0,0,60.0,32400.0,...,0.145281,-0.023683,25.710651,25.710470,25.710616,-89.628639,33.427748,126.662474,0.145418,-0.023663
1,TRAIN_0000001,1,21,0,2,103,0,0,60.0,0.0,...,0.021073,-0.118724,525.890585,525.889848,525.889858,-30.359277,33.502771,126.527673,0.022212,-0.116506
2,TRAIN_0000002,4,7,0,2,103,0,0,80.0,0.0,...,-0.172359,0.081321,608.398971,608.398129,608.398131,-80.243779,33.279609,126.365373,-0.169228,0.082224
3,TRAIN_0000003,0,13,0,2,107,0,0,50.0,0.0,...,0.025040,0.143558,107.352194,107.352052,107.352046,-122.270455,33.245823,126.566716,0.025559,0.143369
4,TRAIN_0000004,6,8,0,2,103,0,0,80.0,0.0,...,-0.178946,-0.103948,337.949009,337.948526,337.948542,81.247401,33.462446,126.328351,-0.180761,-0.103967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,TRAIN_4701212,1,16,0,1,107,0,0,50.0,0.0,...,-0.240548,-0.070376,427.004280,427.003689,427.003691,-108.050664,33.421550,126.275937,-0.238300,-0.070665
4701213,TRAIN_4701213,1,2,0,2,107,0,0,80.0,43200.0,...,-0.083753,-0.100678,48.463884,48.463795,48.463817,87.352999,33.472515,126.424629,-0.084013,-0.100704
4701214,TRAIN_4701214,3,22,0,2,103,0,0,60.0,0.0,...,0.395765,-0.005445,342.184748,342.184270,342.184275,174.259616,33.445652,126.912763,0.395793,-0.006987
4701215,TRAIN_4701215,2,2,0,2,103,0,0,80.0,0.0,...,-0.079176,-0.072250,209.716367,209.716091,209.716077,42.083750,33.444296,126.432574,-0.080023,-0.071661


In [22]:
feature_test

,id,day_of_week,base_hour,road_in_use,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,weight_restricted,...,pca_end_latitude,pca_end_longitude,harversine,cosine,approximation,bearing,m_latitude,m_longitude,m_pca_latitude,m_pca_longitude
0,TEST_000000,1,17,0,3,107,0,0,70.0,0.0,...,0.037947,-0.112305,278.927299,278.926917,278.926914,57.548612,33.500100,126.542567,0.036598,-0.111812
1,TEST_000001,6,12,0,2,103,0,0,70.0,0.0,...,-0.122195,0.110447,1038.940963,1038.939528,1038.939529,-92.375196,33.258313,126.421421,-0.116639,0.111023
2,TEST_000002,0,2,0,1,103,0,0,60.0,0.0,...,-0.063756,0.117462,171.442611,171.442345,171.442375,-80.815071,33.259083,126.475597,-0.062872,0.117709
3,TEST_000003,1,23,0,3,103,0,0,70.0,0.0,...,0.035477,-0.082651,271.087580,271.087207,271.087205,-176.467172,33.472277,126.545557,0.035734,-0.083844
4,TEST_000004,2,17,0,3,106,0,0,70.0,0.0,...,0.074448,-0.103292,1225.872066,1225.870376,1225.870374,114.738656,33.499170,126.575227,0.068819,-0.106402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291236,TEST_291236,5,5,0,1,103,0,0,80.0,0.0,...,-0.048419,-0.093713,329.352532,329.352072,329.352077,-142.045440,33.471651,126.461938,-0.047177,-0.094719
291237,TEST_291237,0,20,0,2,103,0,0,60.0,0.0,...,0.066413,0.093540,524.186844,524.186123,524.186120,165.438411,33.303077,126.599623,0.066024,0.091183
291238,TEST_291238,0,11,0,1,107,0,0,30.0,0.0,...,-0.008948,-0.115510,431.332410,431.331817,431.331814,2.181071,33.495562,126.496858,-0.009302,-0.113602
291239,TEST_291239,0,7,0,2,107,0,0,60.0,0.0,...,-0.031908,0.125463,101.935927,101.935801,101.935786,16.920179,33.255221,126.507174,-0.032126,0.125875


In [23]:
print('------------결측치------------')
display(feature_train.isna().sum())
display(feature_test.isna().sum())

------------결측치------------


id                       0
day_of_week              0
base_hour                0
road_in_use              0
lane_count               0
road_rating              0
multi_linked             0
connect_code             0
maximum_speed_limit      0
weight_restricted        0
road_type                0
start_latitude           0
start_longitude          0
start_turn_restricted    0
end_latitude             0
end_longitude            0
end_turn_restricted      0
target                   0
교통단속수                    0
어린이보호구역수                 0
base_year                0
base_month               0
base_day                 0
base_no_week             0
pca_start_latitude       0
pca_start_longitude      0
pca_end_latitude         0
pca_end_longitude        0
harversine               0
cosine                   0
approximation            0
bearing                  0
m_latitude               0
m_longitude              0
m_pca_latitude           0
m_pca_longitude          0
dtype: int64

id                       0
day_of_week              0
base_hour                0
road_in_use              0
lane_count               0
road_rating              0
multi_linked             0
connect_code             0
maximum_speed_limit      0
weight_restricted        0
road_type                0
start_latitude           0
start_longitude          0
start_turn_restricted    0
end_latitude             0
end_longitude            0
end_turn_restricted      0
교통단속수                    0
어린이보호구역수                 0
base_year                0
base_month               0
base_day                 0
base_no_week             0
pca_start_latitude       0
pca_start_longitude      0
pca_end_latitude         0
pca_end_longitude        0
harversine               0
cosine                   0
approximation            0
bearing                  0
m_latitude               0
m_longitude              0
m_pca_latitude           0
m_pca_longitude          0
dtype: int64

### Save Data

In [24]:
date = str(pd.Timestamp.now())[:10].replace('-','')
feature_train.to_csv(f'../data/feature/{date}_train.csv', index=False)
feature_test.to_csv(f'../data/feature/{date}_test.csv', index=False)